# PVGIS TMY dowload - pvlib PV Generation - pyet Penman Evaporation

Import Libaries

In [110]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gpd
import fiona
import json
import requests
import os
import glob
from tqdm import tqdm

import pvlib
from pvlib.modelchain import ModelChain
from pvlib.location import Location
from pvlib.pvsystem import PVSystem
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
from pvlib import irradiance

In [152]:
from scipy.spatial import cKDTree
from shapely.geometry import Point

In [111]:
def weighted_average(df, values, weights):
    return sum(df[weights] * df[values]) / df[weights].sum()

Import BBDD and filter entries for TMY download

In [134]:
# import of complete database for water bodies in Chile with unique id in column "id"
gdf = gpd.read_file(r'FPV_BBDD_water_bodies.shp') # FPV_BBDD_water_bodies.shp
print("the gdf has "+str(len(gdf))+" entries")
#df = gdf.drop(columns= "geometry")
df = gdf
df["objectid"] = df["objectid"].astype(str)
df.lat_4326 = df.lat_4326.round(4).astype(float)
df.long_4326 = df.long_4326.round(4).astype(float)

# filter only artificial for download
df = df[df["class"] == "artificial"]
print("the gdf has "+str(len(df))+" entries classified as artificial")
# Create small sample
#df= df[df["id"] == 15878] #Embalse Hospital id = 15878
# df = df.sample(n = 25)
df = df.sample(frac=1).drop_duplicates(['REGION'])

# Use only dams for hidroelectric generation
#hidro = pd.read_csv("hidroelectricas.csv", sep=',',encoding='latin-1')
#df = hidro

df = df.reset_index(drop=True)
df.head(3)

the gdf has 30872 entries
the gdf has 3777 entries classified as artificial


,id,objectid,Nombre,Tipo,REGION,area,class,area_real,check,sub_dist_k,hid_dist_k,hid_pot,hid_name,lat_4326,long_4326,geometry
0,24718,AUX-51159,Sin información,TRANQUE,Valparaíso,0.088529,artificial,0.088529,None,14.700927,NaN,NaN,None,-33.8570,-71.6711,"POLYGON ((-7978361.004 -4009617.351, -7978384...."
1,28914,06-2011-07-011,Agrícola Mantos Verdes Limitada,Reparación de embalse de regulación corta Los ...,Maule,0.445120,artificial,0.445120,None,29.998565,NaN,NaN,None,-35.2264,-71.1009,"POLYGON ((-7914874.452 -4194693.209, -7914875...."
2,29771,202-2017-14-040,Margarita Haydée Loncoñanco Santibáñez,Construcción de estanque e instalación de sist...,Los Ríos,0.002400,artificial,0.002400,None,3.472586,NaN,NaN,None,-39.5837,-72.1904,"POLYGON ((-8036198.859 -4805634.991, -8036198...."


Get TMY from PVGIS (with https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.iotools.get_pvgis_tmy.html)

100 items  = 9 min 10 sec

In [135]:
#tmy_all = []
for i in tqdm(df.index):
    try: 
        # API request for points in df
        data_pvgis  = pvlib.iotools.get_pvgis_tmy(df.loc[i,"lat_4326"], df.loc[i,"long_4326"], outputformat='json', usehorizon=True, userhorizon=None, startyear=None, endyear=None, url='https://re.jrc.ec.europa.eu/api/v5_2/', map_variables=None, timeout=30)
    
        # Get altitude and add to df
        df.loc[i,'altitude'] = data_pvgis[2].get("location").get("elevation")

        # Get tmy data 
        tmy_pvg_r = data_pvgis[0]

        # process tmy data: move 3 first rows to back to convert to Chilean time 
        tmy_pvg = tmy_pvg_r.iloc[3:].append(tmy_pvg_r.iloc[:3])
        tmy_pvg["time"] = pd.date_range(start = "2022-01-01 00:00", end="2022-12-31 23:00", freq="h")
        tmy_pvg = tmy_pvg.reset_index(drop=True)

        # Information on PVGIS Download
        if (tmy_pvg["G(h)"].sum() < 1):
            df.loc[i,'PVGIS_dl'] = "missing solar data"
        elif (tmy_pvg["WS10m"].sum() < 1) | (tmy_pvg["T2m"].sum() < 1):
            df.loc[i,'PVGIS_dl'] = "missing climate data"
        else:
            df.loc[i,'PVGIS_dl'] = "ok"

        # add data info
        tmy_pvg["info"] = np.nan
        tmy_pvg["info_values"] = np.nan
        info = df.columns.values.tolist()
        info_values = df.iloc[i]
        for j in range(len(info)):
            tmy_pvg.loc[j,"info"] = info[j]
            tmy_pvg.loc[j,"info_values"] = info_values[j]
            
        # process tmy data: Rename for pvlib
        cols_to_use = ["info", "info_values", "time", "T2m", "G(h)", "Gb(n)", "Gd(h)", "IR(h)", "WS10m", "RH", "SP"]
        pvlib_column_names = ["info", "info_values", "time", "temp_air", "ghi", "dni", "dhi", "lwr_u", "wind_speed", "rh", "sp" ]
        tmy_pvg = tmy_pvg[cols_to_use]
        tmy_pvg.columns = pvlib_column_names

        # Save as csv
        outFileName = "FPV_objectid_" + str(df.loc[i,"id"])
        tmy_pvg.to_csv(outFileName+".csv", sep=',',encoding='latin-1', index=False)

        # Store in List
        #tmy_all.append(tmy_pvg)

    except requests.HTTPError as err:
        df.loc[i,'PVGIS_dl'] = err

print("PVGIS Download finished: "+ str(df[df['PVGIS_dl'] != "ok"].count().id)+ " locations without (complete) TMY data")
df[df['PVGIS_dl'] != "ok"]
df.to_csv("data_used.csv", sep=',',encoding='latin-1', index=False)

  0%|          | 0/16 [00:00<?, ?it/s]c:\Users\david.jung\AppData\Local\Programs\Python\Python39\lib\site-packages\pvlib\iotools\pvgis.py:494: pvlibDeprecationWarning: PVGIS variable names will be renamed to pvlib conventions by default starting in pvlib 0.10.0. Specify map_variables=True to enable that behavior now, or specify map_variables=False to hide this warning.
  warnings.warn(
100%|██████████| 16/16 [03:06<00:00, 11.65s/it]

PVGIS Download finished: 2 locations without (complete) TMY data


,id,objectid,Nombre,Tipo,REGION,area,class,area_real,check,sub_dist_k,hid_dist_k,hid_pot,hid_name,lat_4326,long_4326,geometry,altitude,PVGIS_dl
10,29480,202-2016-12-006,Juana de Lourdes Ruiz Cárcamo,"Construcción de estanque, instalación de siste...",Magallanes y de la Antártica Chilena,0.0002,artificial,0.0002,None,6.445165,NaN,NaN,None,-53.3190,-70.3449,"POLYGON ((-7830757.784 -7042218.791, -7830757....",64.0,missing climate data
12,16218,28090,None,Tranque,Tarapacá,6.2019,artificial,6.2019,None,3.485291,NaN,NaN,None,-20.7818,-70.1719,"POLYGON ((-7811780.748 -2365800.518, -7811228....",NaN,Internal Server Error


Load TMY Data from csv (needs to be in folder PVGIS_TMY)

In [136]:
# Get info on available files
path = os.getcwd()
path = path+"\\PVGIS_TMY_2"
csv_files = glob.glob(path + "\*.csv")

# Filter only small sample
#csv_files = csv_files[:10]

tmy_all = []

# loop over the list of csv files
for f in tqdm(csv_files):
    # read the csv file
    tmy = pd.read_csv(f, encoding='latin-1')
    tmy["time"] = pd.date_range(start = "2022-01-01 00:00", end="2022-12-31 23:00", freq="h")
    tmy.loc[tmy["info"] == "id","info_values"] = pd.to_numeric(tmy.loc[tmy["info"] == "id","info_values"]).astype('int')
    tmy.loc[tmy["info"] == "lat_4326","info_values"] = float(tmy.loc[tmy["info"] == "lat_4326","info_values"])
    tmy.loc[tmy["info"] == "long_4326","info_values"] = float(tmy.loc[tmy["info"] == "long_4326","info_values"])
    tmy.loc[tmy["info"] == "altitude","info_values"] = float(tmy.loc[tmy["info"] == "altitude","info_values"])

    # append tmy's
    tmy_all.append(tmy)

100%|██████████| 15/15 [00:00<00:00, 19.07it/s]


(SKIP) Load TMY Data from csv (needs to be in folder PVGIS_TMY) and create new df based on found TMY

In [94]:
if len(tmy_all) < 1:
    print("No tmy data has been loaded for further analisis")
else:
    # initialize new Dataframe and tmy list
    df = pd.DataFrame(columns=tmy_all[0]["info"][:tmy_all[0]["info"].count()].tolist())

    for i in tqdm(range(0,len(tmy_all))):
        # add info to df
        df.loc[i] = tmy_all[i]["info_values"][:tmy_all[i]["info"].count()].tolist()

df["id"] = pd.to_numeric(df["id"]).astype('int')
df["area_real"] = pd.to_numeric(df["area_real"]).astype('int')

100%|██████████| 16/16 [00:00<00:00, 90.96it/s]


Calculate Evaporation and add Result. Function for Evaporation Calculation based on 
- https://www.nature.com/articles/s41597-021-01003-9#Sec2
- https://github.com/Dagmawi-TA/hPET
- Finch (2001): A comparison between measured and modelled open water evaporation from a reservoir in south‐east England - DOI:10.1002/hyp.267

In [137]:
# Constants
lmbda = 2.45        # Latent heat of vaporization [MJ kg -1] (simplification in the FAO PenMon (latent heat of about 20°C)
cp = 0.001013       # Specific heat at constant pressure [MJ kg-1 °C-1]
eps = 0.622         # Ratio molecular weight of water vapour/dry air
b = 243.04
a = 17.625
eW = 0.98           # emissivity of the lake’s surface (0.98), 
sbc = 4.903*10**-9  # Stefan Bolzmann constant
k = 0.41            # von Karman’s constant 
pa = 1              # density of air

def calculate_ev(
                  sw_down_radiation_W_m2,   # Daily downward shortwave/solar radiation W/m2
                  lw_down_radiation_W_m2,   # Daily downward longwave radiation
                  temperature2m_C,          # Daily average temperature at 2 m
                  temperature2m_C_max,      # Daily max temperature at 2 m
                  temperature2m_C_min,      # Daily min temperature at 2 m
                  rel_hum_p_max,            # Daily Relative humidity max
                  rel_hum_p_min,            # Daily Relative humidity min
                  surface_pressure_Pa,      # surface pressure Pa
                  windspeed10m_m_s,         # Daily average Windspeed at 10 m
                  albedo,                   # Albedo of water
                  heat_stroage             # factor used to get the soil/water heat flux
                  ):               
    """
    This is the function that calculate the PET based on the PM method.
    """
    
    
    # Wind Speed
    windspeed2m_m_s = windspeed10m_m_s*(4.87/(np.log(67.8*10-5.42)))

    # Atmospheric pressure [kPa] eq 7 in FAO.
    P_kPa = surface_pressure_Pa/1000 # 101.3*((293.0-0.0065*height_m) / 293.0)**5.26

    # Psychrometric constant (gamma symbol in FAO) eq 8 in FAO.
    psychometric_kPa_c = cp*P_kPa / (eps*lmbda)

    # Mean Daily temperature as defined by FAO
    temperature2m_C_mean = ( temperature2m_C_max + temperature2m_C_min ) / 2

    # Saturation vapour pressure, eq 11 in FAO.
    svp_kPa = (0.6108*np.exp((17.27*temperature2m_C_max) / (temperature2m_C_max+237.3)) + 0.6108*np.exp((17.27*temperature2m_C_min) / (temperature2m_C_min+237.3)) ) / 2

    # Delta (slope of saturation vapour pressure curve) eq 13 in FAO.
    delta_kPa_C = 4098.0*svp_kPa / (temperature2m_C_mean+237.3)**2

    # Actual vapour pressure, eq 14 in FAO.
    avp_kPa = ( ( 0.6108*np.exp((17.27*temperature2m_C_min) / (temperature2m_C_min+237.3)) * ( rel_hum_p_max / 100 ) ) + ( 0.6108*np.exp((17.27*temperature2m_C_max) / (temperature2m_C_max+237.3)) * ( rel_hum_p_min / 100 ) ) ) /2

    # Saturation vapour pressure deficit.
    svpdeficit_kPa = svp_kPa - avp_kPa

    # Net radiation
    net_sw_radiation_MJ_m2 = (1 - albedo) * sw_down_radiation_W_m2 *0.0036 
    lw_down_radiation_MJ_m2 = lw_down_radiation_W_m2 *0.0036
    out_lw_radiation_MJ_m2 =   0.98 * sbc *  (temperature2m_C+273.3)**4 
    net_lw_radiation_MJ_m2 = lw_down_radiation_MJ_m2 - out_lw_radiation_MJ_m2
    net_radiation_MJ_m2 = net_sw_radiation_MJ_m2 + net_lw_radiation_MJ_m2
   
    # Aerodynamic resistance 
    ra = np.log((2-(2*2/3))/0.001)**2 / (k**2 * windspeed2m_m_s) # -(2*2/3)
    #d = (1 + 4 * sbc * (temperature2m_C +273.3)**4 * ra ) / (86400 * pa * cp)

    
    # Heat Storage [MJ m-2 day-1] - set to 0 following eq 42 in FAO
    N = heat_stroage 
      
    # Calculate Evaporation based on Penman Monteith (1965)
    aero = 86400 * pa * cp *(svpdeficit_kPa) / ra
    numerator = delta_kPa_C*(net_radiation_MJ_m2 - N) + aero
    denominator = ( delta_kPa_C + psychometric_kPa_c ) *lmbda
    Evapo_mm = ( numerator / denominator) 

    df_res = pd.DataFrame()
    

    if lw_down_radiation_W_m2.sum() > 1:
        df_res["ET_mm"] = Evapo_mm
        df_res["rn"] = net_radiation_MJ_m2
    else:
        df_res["ET_mm"] = np.nan
    return df_res

In [138]:
albedo = 0.05
heat_stroage = 0
id_list = []

#Calculate Evaporation
for i in tqdm(range(0,len(tmy_all))):
    if tmy_all[i].loc[tmy_all[i]['info'] == "PVGIS_dl","info_values"].iloc[0] == "ok":
        tmy_all[i].index = pd.date_range(start = "2022-01-01 00:00", end="2022-12-31 23:00", freq="h")
    
        tmy_daily = pd.DataFrame()
        tmy_daily = tmy_all[i][["temp_air", "wind_speed", "rh", "sp"]].resample('D').mean()
        tmy_daily["rh_max"] = tmy_all[i]["rh"].resample('D').max()
        tmy_daily["rh_min"] = tmy_all[i]["rh"].resample('D').min()
        tmy_daily["temp_air_max"] = tmy_all[i]["temp_air"].resample('D').max()
        tmy_daily["temp_air_min"] = tmy_all[i]["temp_air"].resample('D').min()
        tmy_daily["temp_air"] = (tmy_daily["temp_air_min"] +tmy_daily["temp_air_max"] )/2
        tmy_daily["ghi"] = tmy_all[i]["ghi"].resample('D').sum()
        tmy_daily["lwr"] = tmy_all[i]["lwr_u"].resample('D').sum()

        
        ev = calculate_ev(
            tmy_daily["ghi"], 
            tmy_daily["lwr"], 
            tmy_daily["temp_air"], 
            tmy_daily["temp_air_max"],
            tmy_daily["temp_air_min"],
            tmy_daily["rh_max"],
            tmy_daily["rh_min"],
            tmy_daily["sp"],
            tmy_daily["wind_speed"],
            albedo, 
            heat_stroage
            )
             
        tmy_all[i]["ET_mm"] = tmy_all[i].index.map(ev["ET_mm"])
        tmy_all[i]["rn"] = tmy_all[i].index.map(ev["rn"])
        tmy_all[i] = tmy_all[i].reset_index(drop=True)  
      
    
        
        id = math.floor(pd.to_numeric(tmy_all[i].loc[tmy_all[i]["info"] == "id","info_values"].iloc[0]))
        df.loc[(df["id"] == id), "ET_mm/y"] = tmy_all[i]["ET_mm"].sum()
        df.loc[(df["id"] == id), "rn"] = tmy_all[i]["rn"].sum()
        id_list.append(id)


100%|██████████| 15/15 [00:02<00:00,  6.86it/s]


In [139]:
import pyet
def penman_ow(tmean, wind, rs=None, rn=None, g=0, tmax=None, tmin=None,
           rhmax=None, rhmin=None, rh=None, pressure=None, elevation=None,
           lat=None, n=None, nn=None, rso=None, aw=1, bw=0.537, a=1.35,
           b=-0.35, ea=None, albedo=0.23, kab=None, as1=0.25, bs1=0.5,
           ku=6.43, clip_zero=True):
    """Potential evapotranspiration calculated according to
    :cite:t:`penman_natural_1948`.

    Parameters
    ----------
    tmean: pandas.Series/xarray.DataArray
        average day temperature [°C]
    wind: float/pandas.Series/xarray.DataArray
        mean day wind speed [m/s]
    rs: float/pandas.Series/xarray.DataArray, optional
        incoming solar radiation [MJ m-2 d-1]
    rn: float/pandas.Series/xarray.DataArray, optional
        net radiation [MJ m-2 d-1]
    g: float/pandas.Series/xarray.DataArray, optional
        soil heat flux [MJ m-2 d-1]
    tmax: float/pandas.Series/xarray.DataArray, optional
        maximum day temperature [°C]
    tmin: float/pandas.Series/xarray.DataArray, optional
        minimum day temperature [°C]
    rhmax: float/pandas.Series/xarray.DataArray, optional
        maximum daily relative humidity [%]
    rhmin: float/pandas.Series/xarray.DataArray, optional
        mainimum daily relative humidity [%]
    rh: float/pandas.Series/xarray.DataArray, optional
        mean daily relative humidity [%]
    pressure: float/pandas.Series/xarray.DataArray, optional
        atmospheric pressure [kPa]
    elevation: float/xarray.DataArray, optional
        the site elevation [m]
    lat: float/xarray.DataArray, optional
        the site latitude [rad]
    n: float/pandas.Series/xarray.DataArray, optional
        actual duration of sunshine [hour]
    nn: float/pandas.Series/xarray.DataArray, optional
        maximum possible duration of sunshine or daylight hours [hour]
    rso: float/pandas.Series/xarray.DataArray, optional
        clear-sky solar radiation [MJ m-2 day-1]
    aw: float, optional
        wind coefficient [-]
    bw: float, optional
        wind coefficient [-]
    a: float, optional
        empirical coefficient for Net Long-Wave radiation [-]
    b: float, optional
        empirical coefficient for Net Long-Wave radiation [-]
    ea: float/pandas.Series/xarray.DataArray, optional
        actual vapor pressure [kPa]
    albedo: float, optional
        surface albedo [-]
    kab: float, optional
        coefficient derived from as1, bs1 for estimating clear-sky radiation
        [degrees].
    as1: float, optional
        regression constant,  expressing the fraction of extraterrestrial
        reaching the earth on overcast days (n = 0) [-]
    bs1: float, optional
        empirical coefficient for extraterrestrial radiation [-]
    clip_zero: bool, optional
        if True, replace all negative values with 0.

    Returns
    -------
    pandas.Series/xarray.DataArray containing the calculated
            Potential evapotranspiration [mm d-1].

    Examples
    --------
    >>> et_penman = penman(tmean, wind, rn=rn, rh=rh)

    Notes
    -----
    Following :cite:t:`penman_natural_1948` and
    :cite:t:`valiantzas_simplified_2006`.

    .. math:: PET = \\frac{\\frac{\\Delta (R_n-G)}{\\lambda} +
        \\gamma (a_w + b_w u_2) (e_s-e_a)}{(\\Delta +\\gamma)}

    """
    pressure = pyet.calc_press(elevation, pressure)
    gamma = pyet.calc_psy(pressure)
    dlt = pyet.calc_vpc(tmean)
    lambd = pyet.calc_lambda(tmean)

    ea = pyet.calc_ea(tmean=tmean, tmax=tmax, tmin=tmin, rhmax=pyet.check_rh(rhmax),
                 rhmin=pyet.check_rh(rhmin), rh=pyet.check_rh(rh), ea=ea)
    es = pyet.calc_es(tmean=tmean, tmax=tmax, tmin=tmin)

    rn = pyet.calc_rad_net(tmean, rn, rs, pyet.check_lat(lat), n, nn, tmax, tmin, rhmax,
                      rhmin, rh, elevation, rso, a, b, ea, albedo, as1, bs1,
                      kab)
                      

    fu = (aw + bw * wind)

    den = (dlt + gamma)
    num1 = dlt * (rn - g) / den / lambd
    num2 = gamma * (es - ea) * fu / den
    pet = num1 + num2
    pet = pyet.clip_zeros(pet, clip_zero)
    return pet.rename("Penman_ow")

In [140]:
import pyet
def penman_ow_rn(tmean, wind, rs=None, rl=None, rn=None, g=0, tmax=None, tmin=None,
           rhmax=None, rhmin=None, rh=None, pressure=None, elevation=None,
           lat=None, n=None, nn=None, rso=None, aw=1, bw=0.537, a=1.35,
           b=-0.35, ea=None, albedo=0.23, kab=None, as1=0.25, bs1=0.5,
           ku=6.43, clip_zero=True):
    """Potential evapotranspiration calculated according to
    :cite:t:`penman_natural_1948`.

    Parameters
    ----------
    tmean: pandas.Series/xarray.DataArray
        average day temperature [°C]
    wind: float/pandas.Series/xarray.DataArray
        mean day wind speed [m/s]
    rs: float/pandas.Series/xarray.DataArray, optional
        incoming solar radiation [MJ m-2 d-1]
    rn: float/pandas.Series/xarray.DataArray, optional
        net radiation [MJ m-2 d-1]
    g: float/pandas.Series/xarray.DataArray, optional
        soil heat flux [MJ m-2 d-1]
    tmax: float/pandas.Series/xarray.DataArray, optional
        maximum day temperature [°C]
    tmin: float/pandas.Series/xarray.DataArray, optional
        minimum day temperature [°C]
    rhmax: float/pandas.Series/xarray.DataArray, optional
        maximum daily relative humidity [%]
    rhmin: float/pandas.Series/xarray.DataArray, optional
        mainimum daily relative humidity [%]
    rh: float/pandas.Series/xarray.DataArray, optional
        mean daily relative humidity [%]
    pressure: float/pandas.Series/xarray.DataArray, optional
        atmospheric pressure [kPa]
    elevation: float/xarray.DataArray, optional
        the site elevation [m]
    lat: float/xarray.DataArray, optional
        the site latitude [rad]
    n: float/pandas.Series/xarray.DataArray, optional
        actual duration of sunshine [hour]
    nn: float/pandas.Series/xarray.DataArray, optional
        maximum possible duration of sunshine or daylight hours [hour]
    rso: float/pandas.Series/xarray.DataArray, optional
        clear-sky solar radiation [MJ m-2 day-1]
    aw: float, optional
        wind coefficient [-]
    bw: float, optional
        wind coefficient [-]
    a: float, optional
        empirical coefficient for Net Long-Wave radiation [-]
    b: float, optional
        empirical coefficient for Net Long-Wave radiation [-]
    ea: float/pandas.Series/xarray.DataArray, optional
        actual vapor pressure [kPa]
    albedo: float, optional
        surface albedo [-]
    kab: float, optional
        coefficient derived from as1, bs1 for estimating clear-sky radiation
        [degrees].
    as1: float, optional
        regression constant,  expressing the fraction of extraterrestrial
        reaching the earth on overcast days (n = 0) [-]
    bs1: float, optional
        empirical coefficient for extraterrestrial radiation [-]
    clip_zero: bool, optional
        if True, replace all negative values with 0.

    Returns
    -------
    pandas.Series/xarray.DataArray containing the calculated
            Potential evapotranspiration [mm d-1].

    Examples
    --------
    >>> et_penman = penman(tmean, wind, rn=rn, rh=rh)

    Notes
    -----
    Following :cite:t:`penman_natural_1948` and
    :cite:t:`valiantzas_simplified_2006`.

    .. math:: PET = \\frac{\\frac{\\Delta (R_n-G)}{\\lambda} +
        \\gamma (a_w + b_w u_2) (e_s-e_a)}{(\\Delta +\\gamma)}

    """
    pressure = pyet.calc_press(elevation, pressure)
    gamma = pyet.calc_psy(pressure)
    dlt = pyet.calc_vpc(tmean)
    lambd = pyet.calc_lambda(tmean)

    ea = pyet.calc_ea(tmean=tmean, tmax=tmax, tmin=tmin, rhmax=pyet.check_rh(rhmax),
                 rhmin=pyet.check_rh(rhmin), rh=pyet.check_rh(rh), ea=ea)
    es = pyet.calc_es(tmean=tmean, tmax=tmax, tmin=tmin)

    if rl is None:
        rn = pyet.calc_rad_net(tmean, rn, rs, pyet.check_lat(lat), n, nn, tmax, tmin, rhmax,
                      rhmin, rh, elevation, rso, a, b, ea, albedo, as1, bs1,
                      kab)
    else:
        rsn = (1 - albedo) * rs
        rld = rl
        rlo =   0.98 * sbc *  (tmean+273.3)**4 
        rln = rld - rlo
        rn = rsn + rln
    
    fu = (aw + bw * wind)

    den = (dlt + gamma)
    num1 = dlt * (rn - g) / den / lambd
    num2 = gamma * (es - ea) * fu / den
    pet = num1 + num2
    pet = pyet.clip_zeros(pet, clip_zero)
    res_ev = pd.DataFrame()
    res_ev["Penman_ow_rn"] = pet
    res_ev["rn_mod"] = rn
    return res_ev

In [141]:
# Resample and define input meteorological variables
id_list = []

for i in tqdm(range(0,len(tmy_all))):
    
    lat =  pyet.check_lat(pyet.deg_to_rad(tmy_all[i].loc[tmy_all[i]["info"] == "lat_4326","info_values"])).values[0]
    ele = tmy_all[i].loc[tmy_all[i]["info"] == "altitude","info_values"].values[0]

    tmy_all[i].index = pd.date_range(start = "2022-01-01 00:00", end="2022-12-31 23:00", freq="h")
    tmax = tmy_all[i]["temp_air"].resample("D").max()
    tmin = tmy_all[i]["temp_air"].resample("D").min()
    tmean = ( tmax + tmin ) / 2

    rhmax = tmy_all[i]["rh"].resample("D").max()
    rhmin = tmy_all[i]["rh"].resample("D").min()
    rh = tmy_all[i]["rh"].resample("D").mean()

    wind = tmy_all[i]["wind_speed"].resample("D").mean()

    rs = tmy_all[i]["ghi"].resample("D").sum() * 0.0036
    lwd = tmy_all[i]["lwr_u"].resample("D").sum() * 0.0036

    lambda0 = 2.45  
    lambda1 = pyet.calc_lambda(tmean)
    lambda_cor = lambda1 / lambda0
    aw = 1.313 
    bw = 1.381 
    if tmy_all[i].loc[tmy_all[i]['info'] == "PVGIS_dl","info_values"].iloc[0] == "ok":
        ev_pm = penman_ow(tmean, wind=wind, rs=rs, tmax=tmax, tmin=tmin, rh=rh, rhmin=rhmin, rhmax=rhmax, elevation=ele, lat=lat, aw=aw, bw=bw, albedo=0.05) / lambda_cor
        tmy_all[i]["ET_mm_pyet"] = tmy_all[i].index.map(ev_pm)
        rn_pyet = pyet.calc_rad_net(tmean, rs=rs, tmax=tmax, tmin=tmin, rh=rh, rhmin=rhmin, rhmax=rhmax, elevation=ele, lat=lat, albedo=0.05)
        tmy_all[i]["rn_pyet"] = tmy_all[i].index.map(rn_pyet)
        

        ev_pm_rn = penman_ow_rn(tmean, wind=wind, rs=rs, rl=lwd, tmax=tmax, tmin=tmin, rh=rh, rhmin=rhmin, rhmax=rhmax, elevation=ele, lat=lat, aw=aw, bw=bw, albedo=0.05)["Penman_ow_rn"] / lambda_cor
        tmy_all[i]["ET_mm_pyet_mod"] = tmy_all[i].index.map(ev_pm_rn)
        rn_pyet_mod = penman_ow_rn(tmean, wind=wind, rs=rs, rl=lwd, tmax=tmax, tmin=tmin, rh=rh, rhmin=rhmin, rhmax=rhmax, elevation=ele, lat=lat, aw=aw, bw=bw, albedo=0.05)["rn_mod"]
        tmy_all[i]["rn_pyet_mod"] = tmy_all[i].index.map(rn_pyet_mod)


        # Write results in dataframe
        id = math.floor(pd.to_numeric(tmy_all[i].loc[tmy_all[i]["info"] == "id","info_values"].iloc[0]))
        df.loc[(df["id"] == id), "ET_mm/y_pyet"] = tmy_all[i]["ET_mm_pyet"].sum()
        df.loc[(df["id"] == id), "rn_pyet"] = tmy_all[i]["rn_pyet"].sum()
        df.loc[(df["id"] == id), "ET_mm/y_pyet_mod"] = tmy_all[i]["ET_mm_pyet_mod"].sum()
        df.loc[(df["id"] == id), "rn_pyet_mod"] = tmy_all[i]["rn_pyet_mod"].sum()        
        id_list.append(id)
    tmy_all[i] = tmy_all[i].reset_index(drop=True) 


100%|██████████| 15/15 [00:04<00:00,  3.26it/s]


In [142]:
df.head(5)

,id,objectid,Nombre,Tipo,REGION,area,class,area_real,check,sub_dist_k,...,long_4326,geometry,altitude,PVGIS_dl,ET_mm/y,rn,ET_mm/y_pyet,rn_pyet,ET_mm/y_pyet_mod,rn_pyet_mod
0,24718,AUX-51159,Sin información,TRANQUE,Valparaíso,0.088529,artificial,0.088529,None,14.700927,...,-71.6711,"POLYGON ((-7978361.004 -4009617.351, -7978384....",39.0,ok,1694.660119,4704.757346,1844.167247,5183.162466,1723.979283,4704.757346
1,28914,06-2011-07-011,Agrícola Mantos Verdes Limitada,Reparación de embalse de regulación corta Los ...,Maule,0.445120,artificial,0.445120,None,29.998565,...,-71.1009,"POLYGON ((-7914874.452 -4194693.209, -7914875....",526.0,ok,1434.871491,4236.133604,1654.143489,4893.144213,1489.777981,4236.133604
2,29771,202-2017-14-040,Margarita Haydée Loncoñanco Santibáñez,Construcción de estanque e instalación de sist...,Los Ríos,0.002400,artificial,0.002400,None,3.472586,...,-72.1904,"POLYGON ((-8036198.859 -4805634.991, -8036198....",254.0,ok,957.301821,3355.651927,1097.321271,3831.987026,986.160121,3355.651927
3,30525,14-2020-16-003,Comunidad de Aguas Canal Alimentador Las Patag...,Rehabilitación de embalse de regulación corta,Ñuble,0.305272,artificial,0.305272,None,20.999758,...,-71.8300,"POLYGON ((-7996040.048 -4382111.121, -7996040....",229.0,ok,1470.468032,4069.126562,1664.555691,4669.229556,1516.615879,4069.126562
4,30312,14-2019-10-009,Helen Vanessa Nenen Manriquez,Instalación de sistema scalls e instalación de...,Los Lagos,0.001000,artificial,0.001000,None,17.345720,...,-73.6400,"POLYGON ((-8197565.071 -5141525.922, -8197565....",40.0,ok,1072.425099,3339.534719,1165.988511,3715.165403,1082.455238,3339.534719


PV Generation with pvlib using PV Systs cell temperature and PV Watts DC/AC Generation

In [143]:
albedo = 0.05          # albedo is already defined in Evaporation calculation 

# PV module setup and characteristics 
tilt = 10               # tilt of PV panel in degrees [°]
azimuth = 0             # orientation of tilted pv paneles in degrees [°] with 0° equaling north, 90°

# PV Syst temperature model
module_eff = 0.21
u_c = 25
u_v = 6.84                # Combined heat loss factor coefficient. The default value (29) is representative of freestanding modules with the rear surfaces exposed to open air (e.g., rack mounted).
u_c_fpv = 35.3            # Combined heat loss factor coefficient. The default value (29) is representative of freestanding modules with the rear surfaces exposed to open air (e.g., rack mounted).
u_v_fpv = 8.9                 # Combined heat loss factor influenced by wind

# PV WATTS
pdc0_m = 1000           # Power of the modules at 1000 W/m^2 and cell reference temperature. [W]
gamma_pdc = -0.37       # The temperature coefficient of power. Typically -0.002 to -0.005 per degree C. [1/C]

pdc0_i = 1000           # DC input limit of the inverter [W]
eta_inv_nom=0.961       # Nominal inverter efficiency, default is 0.9637 [unitless] 

#  Iteration to calculate PV Generation 
for i in tqdm(range(0,len(tmy_all))):
    if tmy_all[i].loc[tmy_all[i]['info'] == "PVGIS_dl","info_values"].iloc[0] == "ok":
        # get parameters
        id = math.floor(pd.to_numeric(tmy_all[i].loc[tmy_all[i]["info"] == "id","info_values"].iloc[0]))
        lat = float(tmy_all[i].loc[tmy_all[i]["info"] == "lat_4326" ,"info_values"])
        lon = float(tmy_all[i].loc[tmy_all[i]["info"] == "long_4326","info_values"])
        alt = float(tmy_all[i].loc[tmy_all[i]["info"] == "altitude","info_values"])

        # set index
        tmy_all[i].index = pd.date_range(start = "2022-01-01 00:00", end="2022-12-31 23:00", freq="h")    

        # Get solar azimuth and zenith to pass to the transposition function
        location = Location(
            latitude = lat, 
            longitude = lon, 
            tz = 'America/Santiago', 
            altitude = alt)
        solar_position = location.get_solarposition(times=tmy_all[i]["time"])

        # Use the get_total_irradiance function to transpose the GHI to POA
        POA_irradiance = pvlib.irradiance.get_total_irradiance(
            surface_tilt=tilt,
            surface_azimuth=azimuth,
            solar_zenith = solar_position['apparent_zenith'],
            solar_azimuth = solar_position['azimuth'],
            dni=tmy_all[i]['dni'],
            ghi=tmy_all[i]['ghi'],
            dhi=tmy_all[i]['dhi'],
            dni_extra=None, 
            airmass=None, 
            albedo=albedo, 
            model='isotropic')

        # Calculate effective Irradiance 
        aoi = pvlib.irradiance.aoi(
            surface_tilt=tilt, 
            surface_azimuth=azimuth, 
            solar_zenith=solar_position.apparent_zenith, 
            solar_azimuth=solar_position.azimuth )
        iam = pvlib.iam.ashrae(aoi)
        effective_irradiance = POA_irradiance["poa_direct"]*iam + POA_irradiance["poa_diffuse"]

        # Modeling using the PVsyst model for cell temperature
        temp_cell = pvlib.temperature.pvsyst_cell(
            POA_irradiance["poa_global"], 
            tmy_all[i]["temp_air"], 
            tmy_all[i]["wind_speed"], 
            u_c=u_c, 
            u_v=u_v, 
            eta_m=0.1, 
            alpha_absorption=0.9)  # must be equal to (1 - module_efficiency)?

        result_dc = pvlib.pvsystem.pvwatts_dc(
            effective_irradiance, 
            temp_cell, 
            pdc0_m, 
            gamma_pdc/100, 
            temp_ref=25)

        results_ac = pvlib.inverter.pvwatts(
            pdc=result_dc, 
            pdc0=pdc0_i, 
            eta_inv_nom=eta_inv_nom, 
            eta_inv_ref=0.9637)

        losses = pvlib.pvsystem.pvwatts_losses(
            soiling=5, 
            shading=3, 
            snow=0, 
            mismatch=2, 
            wiring=2, 
            connections=0.5, 
            lid=1.5, 
            nameplate_rating=1, 
            age=0, 
            availability=3)

        results_ac_real = results_ac * (1-losses/100)

        temp_cell_fpv = pvlib.temperature.pvsyst_cell(
            POA_irradiance["poa_global"], 
            tmy_all[i]["temp_air"], 
            tmy_all[i]["wind_speed"], 
            u_c=u_c_fpv, 
            u_v=u_v_fpv, 
            eta_m=0.1, 
            alpha_absorption=0.9)  # must be equal to (1 - module_efficiency)?

        result_dc_fpv = pvlib.pvsystem.pvwatts_dc(
            effective_irradiance, 
            temp_cell_fpv, 
            pdc0_m, 
            gamma_pdc/100, 
            temp_ref=25)

        results_ac_fpv = pvlib.inverter.pvwatts(
            pdc=result_dc_fpv, 
            pdc0=pdc0_i, 
            eta_inv_nom=eta_inv_nom, 
            eta_inv_ref=0.9637)

            
        results_ac_real_fpv = results_ac_fpv * (1-losses/100)

        # Add results to tmy DataFrame
        tmy_all[i]["pv_ac_W"] = results_ac_real
        tmy_all[i]["pv_ac_W_fpv"] = results_ac_real_fpv
        tmy_all[i] = tmy_all[i].reset_index(drop=True)

        # Add results to water bodies DataFrame
        df.loc[(df["id"] == id), "pv_ac_kWh/kWp/y"] = tmy_all[i].pv_ac_W.sum()/1000
        df.loc[(df["id"] == id), "pv_ac_fpv_kWh/kWp/y"] = tmy_all[i].pv_ac_W_fpv.sum()/1000

        # Save result as csv
        id = math.floor(pd.to_numeric(tmy_all[i].loc[tmy_all[i]["info"] == "id","info_values"].iloc[0]))
        outFileName = "FPV_objectid_" + str(id) + "_result"
        tmy_all[i].to_csv(outFileName+".csv", sep=',',encoding='latin-1', index=False)

  0%|          | 0/15 [00:00<?, ?it/s]c:\Users\david.jung\AppData\Local\Programs\Python\Python39\lib\site-packages\pvlib\temperature.py:381: pvlibDeprecationWarning: 
eta_m overwriting module_efficiency
  warn_deprecated(
100%|██████████| 15/15 [00:05<00:00,  3.00it/s]


In [166]:
# Back up geometry column and create centroid column
df["geometry_save"] = df["geometry"]
df["geometry"] = df["geometry"].centroid

# split df in results and errors
df_results = df[df["PVGIS_dl"] == "ok"]
df_errors = df[df["PVGIS_dl"] != "ok"]

# Checknearest function (https://gis.stackexchange.com/questions/222315/finding-nearest-point-in-other-geodataframe-using-geopandas)
def ckdnearest(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdB_nearest = gdB_nearest.add_suffix('_n')
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)

    return gdf

# apply checknearest function
df_errors = ckdnearest(df_errors, df_results)

# Restore original df geometry columns and add missing information
df["geometry"] = df["geometry_save"]
df = df.drop(columns="geometry_save")
for ind in tqdm(df_errors.index):
    id = df_errors.loc[ind, 'id']
    df.loc[(df["id"] == id),"ET_mm/y"] = df_errors.loc[(df_errors["id"] == id),"ET_mm/y_n"].iloc[0]
    df.loc[(df["id"] == id),"ET_mm/y_pyet_mod"] = df_errors.loc[(df_errors["id"] == id),"ET_mm/y_pyet_mod_n"].iloc[0]
    df.loc[(df["id"] == id),"pv_ac_kWh/kWp/y"] = df_errors.loc[(df_errors["id"] == id),"pv_ac_kWh/kWp/y_n"].iloc[0]
    df.loc[(df["id"] == id),"pv_ac_fpv_kWh/kWp/y"] = df_errors.loc[(df_errors["id"] == id),"pv_ac_fpv_kWh/kWp/y_n"].iloc[0]

,id,objectid,Nombre,Tipo,REGION,area,class,area_real,check,sub_dist_k,...,ET_mm/y_n,rn_n,ET_mm/y_pyet_n,rn_pyet_n,ET_mm/y_pyet_mod_n,rn_pyet_mod_n,pv_ac_kWh/kWp/y_n,pv_ac_fpv_kWh/kWp/y_n,geometry_save_n,dist
0,29480,202-2016-12-006,Juana de Lourdes Ruiz Cárcamo,"Construcción de estanque, instalación de siste...",Magallanes y de la Antártica Chilena,0.0002,artificial,0.0002,None,6.445165,...,674.619125,2798.096946,787.866823,3265.472322,702.106216,2798.096946,906.166496,916.274828,POINT (-8078123.530 -5980800.643),1.089861e+06
1,16218,28090,None,Tranque,Tarapacá,6.2019,artificial,6.2019,None,3.485291,...,2511.911147,5391.912570,2542.004340,6013.631099,2378.540695,5391.912570,1471.432492,1484.794967,POINT (-7668387.857 -2565368.130),2.455159e+05


In [ ]:
df.to_csv("FPV_Results_PV_complete.csv", sep=',',encoding='latin-1', index=False)

FPV Impact calculation

In [ ]:
df = pd.read_csv("FPV_Results_PV_complete.csv", sep=',',encoding='latin-1')

In [177]:
# FPV integration parameters
cr_big_c = 0.15      # Lake Area covered by FPV
cr_big_o = 0.4
cr_small_c = 0.4
cr_small_o = 0.6
sizegreatlake = 1 
density = 1.2           # MW/ha
fpv_evapo = 0.6         # Factor for reduction of evaporation 

# LCOE calculation
capex_fpv_utility = 835000      # per MWp
capex_fpv_PMGD = 992000        # per MWp
capex_fpv_nb = 1236000          # per MWp
conection = 40000               # /km2
opex_fpv = 19000                # per MWp/y
wacc = 0.055
degre = 0.005
inflation = 0.03
N = 25


cr_big = cr_big_o
cr_small = cr_small_o

for ind in tqdm(df.index):
    df.loc[ind,'area_FPV'] = np.where(df.loc[ind, "area_real"] > sizegreatlake,  df.loc[ind, "area_real"]* cr_big, df.loc[ind, "area_real"]* cr_small)
    df.loc[ind,'MWp'] = df.loc[ind, "area_FPV"] * density
    df.loc[ind,'category'] = np.where(df.loc[ind, "MWp"] > 3, np.where(df.loc[ind, "MWp"] < 9, "PMGD", "utility"),"net-billing")
    df.loc[ind,'MWh/a'] = df.loc[ind,'MWp'] * df.loc[ind,'pv_ac_kWh/kWp/y']
    df.loc[ind,'ET_m3_total'] = df.loc[ind,'area_real'] * 10000 * (df.loc[ind, 'ET_mm/y'] / 1000)
    df.loc[ind,'ET_m3_saved'] = df.loc[ind,'area_FPV'] * 10000 * (df.loc[ind,'ET_mm/y'] / 1000) * fpv_evapo
    df.loc[ind, "CAPEX"] = np.where(df.loc[ind,'category'] == "utility", capex_fpv_utility * df.loc[ind,"MWp"] + df.loc[ind,'sub_dist_k'] * conection,
      np.where(df.loc[ind,'category'] == "PMGD", capex_fpv_PMGD * df.loc[ind,"MWp"] + df.loc[ind,'sub_dist_k'] * conection, capex_fpv_nb * df.loc[ind,"MWp"]))
    df.loc[ind, "%_capex_conection"] = np.where(df.loc[ind,'category'] != "net-billing", round(1/df.loc[ind, "CAPEX"]*df.loc[ind,'sub_dist_k'] * conection,2)," ")
    cashflow = pd.DataFrame(index=range(0,N))
    cashflow["OPEX_des"] = (opex_fpv * df.loc[ind,"MWp"] * (1+inflation)**cashflow.index) / (1+wacc)**cashflow.index
    cashflow["EG_des"] = (df.loc[ind,"MWh/a"] * (1-degre)**cashflow.index) / (1+wacc)**cashflow.index

    df.loc[ind,"LCOE"] =  (df.loc[ind, "CAPEX"] + cashflow["OPEX_des"].sum() ) / cashflow["EG_des"].sum()

df["ET_mm/y"] = np.where((df["area_FPV"]>0) & (df["ET_mm/y"]<1),np.nan,df["ET_mm/y"])
df.head(5)


100%|██████████| 16/16 [00:00<00:00, 69.30it/s]


Analisis of Data

In [179]:
df_a = df
analisis = df_a.groupby('REGION')['area_real', 'area_FPV', 'MWp', "MWh/a", "ET_m3_saved"].sum()
analisis.loc["Chile total/mean"] =  analisis.sum()
analisis_m = df_a.groupby('REGION')['pv_ac_kWh/kWp/y', 'pv_ac_fpv_kWh/kWp/y', 'ET_mm/y'].mean()
analisis_m.loc["Chile total/mean"] =  analisis_m.mean()
analisis = analisis.join(analisis_m)

analisis_wm = df_a.groupby('REGION').apply(weighted_average, 'LCOE', 'MWh/a')
analisis_wm.loc["Chile total/mean"] =  analisis_wm.mean()
analisis["LCOE"] = analisis_wm
analisis_c = df_a.groupby('REGION')['area_real'].count()
analisis["count"] = analisis_c


analisis = analisis.reindex(['Arica y Parinacota', 'Tarapacá', 'Antofagasta', 'Atacama', 'Coquimbo', 'Valparaíso', 'Metropolitana de Santiago', "O'Higgins", 'Maule', 'Ñuble', 'Biobío', 'La Araucanía', 'Los Ríos', 'Los Lagos', 'Aysén del General Carlos Ibáñez del Campo','Magallanes y de la Antártica Chilena', "Chile total/mean"])

analisis = analisis.rename(columns={"area_real": "artificial_lakes_ha", "area_FPV":"FPV_covered_ha","MWp": "FPV_capacidad_MWp", "pv_ac_kWh/kWp/y": "FPV_kWh/kWp/y", "pv_ac_fpv_kWh/kWp/y": "FPV_cooling_kWh/kWp/y", "MWh/a":"FPV_total_GWh/a"})
analisis["FPV_total_GWh/a"] = analisis["FPV_total_GWh/a"]/1000
analisis["ET_m3_saved"] = analisis["ET_m3_saved"]
#analisis = analisis.fillna(0)
analisis = analisis.round(2)
analisis = analisis[['count', 'artificial_lakes_ha', 'FPV_covered_ha', 'FPV_capacidad_MWp', 'FPV_kWh/kWp/y', 'FPV_cooling_kWh/kWp/y', 'FPV_total_GWh/a', 'LCOE', 'ET_mm/y', 'ET_m3_saved' ]]
analisis.head(20)

C:\Users\david.jung\AppData\Local\Temp\ipykernel_6300\1426451489.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  analisis = df_a.groupby('REGION')['area_real', 'area_FPV', 'MWp', "MWh/a", "ET_m3_saved"].sum()
C:\Users\david.jung\AppData\Local\Temp\ipykernel_6300\1426451489.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  analisis_m = df_a.groupby('REGION')['pv_ac_kWh/kWp/y', 'pv_ac_fpv_kWh/kWp/y', 'ET_mm/y'].mean()


,count,artificial_lakes_ha,FPV_covered_ha,FPV_capacidad_MWp,FPV_kWh/kWp/y,FPV_cooling_kWh/kWp/y,FPV_total_GWh/a,LCOE,ET_mm/y,ET_m3_saved
REGION,,,,,,,,,,
Arica y Parinacota,1.0,0.00,0.00,0.00,1577.47,1596.07,0.00,74.93,1720.95,3.72
Tarapacá,1.0,6.20,2.48,2.98,1471.43,1484.79,4.38,80.33,2511.91,37388.69
Antofagasta,1.0,0.01,0.01,0.01,1471.43,1484.79,0.01,80.33,2511.91,107.79
Atacama,1.0,0.20,0.12,0.14,1360.11,1376.64,0.19,86.91,1841.21,1310.64
Coquimbo,1.0,0.01,0.01,0.01,1280.21,1297.09,0.01,92.33,1430.02,64.87
Valparaíso,1.0,0.09,0.05,0.06,1262.43,1273.49,0.08,93.63,1694.66,540.10
Metropolitana de Santiago,1.0,0.40,0.24,0.28,1183.13,1198.84,0.34,99.91,1262.35,1795.36
O'Higgins,1.0,8.81,3.52,4.23,1218.75,1233.77,5.15,92.30,1515.30,32031.95
Maule,1.0,0.45,0.27,0.32,1174.63,1189.41,0.38,100.63,1434.87,2299.28


In [94]:
analisis.to_csv("FPV_Results_op.csv", sep=',',encoding='latin-1')

In [52]:
analisis = df_a.groupby('REGION')['area_real', 'area_FPV', 'MWp', "MWh/a", "ET_m3_saved"].sum()
analisis.loc["Chile total/mean"] =  analisis.sum()
analisis_m = df_a.groupby('REGION')['pv_ac_kWh/kWp/y', 'pv_ac_fpv_kWh/kWp/y', 'ET_mm/y'].mean()
analisis_m.loc["Chile total/mean"] =  analisis_m.mean()
analisis = analisis.join(analisis_m)

analisis_wm = df_a.groupby('REGION').apply(weighted_average, 'LCOE', 'MWh/a')
analisis_wm.loc["Chile total/mean"] =  analisis_wm.mean()
analisis["LCOE"] = analisis_wm


analisis = analisis.reindex(['Arica y Parinacota', 'Tarapacá', 'Antofagasta', 'Atacama', 'Coquimbo', 'Valparaíso', 'Metropolitana de Santiago', "Libertador General Bernardo O'Higgins", 'Maule', 'Ñuble', 'Biobío', 'La Araucanía', 'Los Ríos', 'Los Lagos', 'Aysén del General Carlos Ibáñez del Campo','Magallanes y de la Antártica Chilena', "Chile total/mean"])

analisis = analisis.rename(columns={"area_real": "artificial_lakes_ha", "area_FPV":"FPV_covered_ha","MWp": "FPV_capacidad_MWp", "pv_ac_kWh/kWp/y": "FPV_kWh/kWp/y", "pv_ac_fpv_kWh/kWp/y": "FPV_cooling_kWh/kWp/y", "MWh/a":"FPV_total_GWh/a"})
analisis["FPV_total_GWh/a"] = analisis["FPV_total_GWh/a"]/1000
analisis["ET_m3_saved"] = analisis["ET_m3_saved"]
#analisis = analisis.fillna(0)
analisis = analisis.round(2)
analisis = analisis[['artificial_lakes_ha', 'FPV_covered_ha', 'FPV_capacidad_MWp', 'FPV_kWh/kWp/y', 'FPV_cooling_kWh/kWp/y', 'FPV_total_GWh/a', 'LCOE', 'ET_mm/y', 'ET_m3_saved' ]]
analisis.head(20)

C:\Users\david.jung\AppData\Local\Temp\ipykernel_6300\1119258378.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  analisis = df_a.groupby('REGION')['area_real', 'area_FPV', 'MWp', "MWh/a", "ET_m3_saved"].sum()
C:\Users\david.jung\AppData\Local\Temp\ipykernel_6300\1119258378.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  analisis_m = df_a.groupby('REGION')['pv_ac_kWh/kWp/y', 'pv_ac_fpv_kWh/kWp/y', 'ET_mm/y'].mean()


,artificial_lakes_ha,FPV_covered_ha,FPV_capacidad_MWp,FPV_kWh/kWp/y,FPV_cooling_kWh/kWp/y,FPV_total_GWh/a,LCOE,ET_mm/y,ET_m3_saved
REGION,,,,,,,,,
Arica y Parinacota,0.01,0.01,0.01,1529.00,1547.41,0.02,77.31,1865.09,91.81
Tarapacá,0.00,0.00,0.00,1591.50,1608.38,0.00,74.27,2136.42,9.23
Antofagasta,0.04,0.03,0.03,1462.87,1480.84,0.04,80.80,2206.30,336.77
Atacama,0.21,0.13,0.15,1352.11,1368.76,0.21,87.42,1791.18,1364.82
Coquimbo,0.03,0.02,0.02,1293.17,1310.89,0.02,91.41,1470.53,137.85
Valparaíso,3.02,1.21,1.45,1271.12,1285.38,1.84,92.99,1538.90,11161.94
Metropolitana de Santiago,0.54,0.33,0.39,1213.77,1230.20,0.47,97.38,1401.21,2741.53
Libertador General Bernardo O'Higgins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Maule,0.00,0.00,0.00,1196.50,1211.00,0.00,98.79,1492.71,10.75


In [63]:
analisis.to_csv("FPV_Results_o.csv", sep=',',encoding='latin-1')

# BACKUP

Store data in csv

In [ ]:
df.to_csv("FPV_results_final.csv", sep=',', encoding='latin-1', index=False)
df_FPV = df[df["ET_mm/y"] > 0]
df_FPV.to_csv("FPV_results_final_only_potential.csv", sep=',', encoding='latin-1', index=False)
df_FPV = df_FPV.reset_index(drop=True)
df = df_FPV

Load Results

In [ ]:
tmy_all = []
path = os.getcwd()
path = path+"\\results"
csv_files = glob.glob(path + "\*.csv")

# loop over the list of csv files
for f in tqdm(csv_files):
    # read the csv file
    tmy = pd.read_csv(f, encoding='latin-1')
    tmy["time"] = pd.date_range(start = "2022-01-01 00:00", end="2022-12-31 23:00", freq="h")
    tmy.loc[8,"info_values"] = float(tmy.loc[8,"info_values"])
    tmy.loc[9,"info_values"] = float(tmy.loc[9,"info_values"])
    tmy.loc[10,"info_values"] = float(tmy.loc[10,"info_values"])

    # append
    tmy_all.append(tmy)

In [353]:
fig = px.scatter_geo(df_FPV, lat=df_FPV['lat_4326'], lon=df_FPV['long_4326'], color_continuous_scale="LCOE",
                     hover_name="Nombre", size="MWp", projection="natural earth", scope='south america')
fig.show()